# Classifying high dimensionality remote sensing images

## Intro to hyperspectral remote sensing
Insert graphic of cube, explanative text

In [1]:
from arcgis.gis import GIS
from arcgis import raster

In [2]:
gis = GIS("https://dev0001902.esri.com/portal","admin", verify_cert=False)

Enter password: ········


In [3]:
aviris_item = gis.content.get('596098ec0e834e7d9f8e81e14de14ef2')
aviris_item

<Item title:"omaha_aviris_wv" type:Imagery Layer owner:admin>

In [5]:
map1 = gis.map('Omaha, NE')
map1

In [6]:
map1.add_layer(aviris_item)

## Draw using false color composite

### Get AVIRIS band wavelengths

In [4]:
aviris_layer = aviris_item.layers[0]
aviris_bands = aviris_layer.key_properties()['BandProperties']
len(aviris_bands)

224

In [5]:
aviris_bands[0]['wavelength']

'365.9298'

In [6]:
aviris_bands[-1]['wavelength']

'2497.036'

In [7]:
aviris_layer = aviris_item.layers[0]
aviris_fcc = raster.stretch(raster = raster.extract_band(aviris_layer, band_wavelengths=[831, 665.0994, 560]),
                            stretch_type='histogram')
map1.remove_layers()
map1.add_layer(aviris_fcc)

NameError: name 'map1' is not defined

## Get spectral signature from the remotely sensed image

In [8]:
map2 = gis.map('Omaha, NE', zoomlevel=10)
map2

In [9]:
map2.add_layer(aviris_fcc)

In [19]:
x_axis = [float(s['wavelength']) for s in aviris_bands]

from bokeh.models import Range1d
from bokeh.plotting import figure, show, output_notebook
from IPython.display import clear_output
output_notebook()

Loading BokehJS ...

In [21]:
def get_samples(map_handle, clicked_point):
    image_data = aviris_layer.get_samples(clicked_point)
    clear_output()
    map_handle.draw(clicked_point)
    radiance_values = image_data[0]['value']
   
    p = figure(title="Spectral Profile", x_axis_label='Wavelengths', 
               y_axis_label='Radiance', width=600, height=300)
    p.line(x_axis, radiance_values, legend="Selected Point", line_color="red", line_width=2)
    p.circle(x_axis, radiance_values, line_color="red", fill_color="white", size=8)
#     p.y_range=Range1d(0, 1.0)
    show(p)
    
map2.on_click(get_samples)

g={'type': 'point', 'x': -10739095.918328783, 'y': 5030225.212874767, 'spatialReference': {'wkid': 102100, 'latestWkid': 3857}}
samples = aviris_layer.get_samples(g)
samples